In [ ]:
import geopandas as gpd
from matplotlib.cbook import pts_to_midstep
from networkx import neighbors
from numpy.ma.core import append

gpd.read_file('blocks (1).geojson')

In [ ]:
# Пример вызова генплана на большой территории 

import geopandas as gpd
from shapely import Polygon, MultiPolygon

from app.gen_planner.python.src import GenPlanner, gen_plan, basic_func_zone

zones, roads = GenPlanner(gpd.read_file('1.gpkg')).poly2func2terr2block(gen_plan)

In [ ]:
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', categorical=True)
m1

In [ ]:
# тут вызовы по сценариям по разным территориям, везде basic по умолчанию, но другие лежат в zoning.


from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('Аэродром_лодейнопольское_поселение.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner, basic_func_zone
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2terr2block(basic_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
# тут вызов деления территориальной зоны на кварталы по минимальному размеру терр.зоны
from app.gen_planner.python.src import GenPlanner, terr_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).poly2block(terr_zones.residential_terr)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
from app.gen_planner.python.src import GenPlanner
import geopandas as gpd

zones = {'1': 0.1,
         '2': 0.1,
         '3': 0.1,
         '4': 0.1,
         '5': 0.1,
         '6': 0.1,
         '7': 0.1,
         '8': 0.1, }
zones, roads = GenPlanner(gpd.read_file('светогорск.geojson')).split_poly(zones, roads_width=50)
zones.explore()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely import MultiPolygon, Polygon

local_crs = 32636
multip = gpd.read_file('2.gpkg').to_crs(local_crs)


def elasatic_wrap(gdf: gpd.GeoDataFrame) -> Polygon:
    gdf = gdf.copy()
    multip = gpd.GeoDataFrame(geometry=gdf.geometry, crs=gdf.crs).explode(ignore_index=True)
    max_dist = np.ceil(multip.apply(lambda row: multip.drop(row.name).distance(row.geometry).min(), axis=1).max(axis=0))
    poly = multip.buffer(max_dist).union_all().buffer(-max_dist)
    if isinstance(poly, MultiPolygon):
        raise RuntimeError('Cant cast Multipolygon to Polygon')
    poly = Polygon(poly.exterior)
    return poly


p = elasatic_wrap(multip)

In [ ]:
gpd.GeoDataFrame(geometry=[p.simplify(250)], crs=local_crs).explore()

In [ ]:
from app.gen_planner.python.src import GenPlanner, gen_plan
import geopandas as gpd

gp = GenPlanner(gpd.read_file('2.gpkg'), rotation=0)
multiterritory = gp.transformed_poly
zones, roads = gp.poly2func2terr2block(gen_plan)

In [ ]:
zones.explore()

# MultiP

In [1]:
import geopandas as gpd
from app.gen_planner.python.src.utils.geom_utils import geometry_to_multilinestring
from shapely.ops import polygonize, unary_union, Polygon


def territory_splitter(gdf_to_split: gpd.GeoDataFrame, splitters: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    local_crs = gdf_to_split.estimate_utm_crs()
    gdf_to_split = gdf_to_split.to_crs(local_crs)
    splitters = splitters.to_crs(local_crs)
    lines_orig = gdf_to_split.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()
    lines_splitters = splitters.to_crs(local_crs).geometry.apply(geometry_to_multilinestring).to_list()

    polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_splitters))),
                                crs=local_crs).clip(gdf_to_split.to_crs(local_crs), keep_geom_type=True).explode()

    polygons_points = polygons.copy()
    polygons_points.geometry = polygons.representative_point()
    joined_ind = polygons_points.sjoin(splitters, how='inner', predicate='within').index.tolist()
    polygons['is_splitter'] = polygons.index.isin(joined_ind)
    polygons.geometry = polygons.apply(lambda x: Polygon(x.geometry.exterior) if not x['is_splitter'] else x.geometry,
                                       axis=1)
    non_splitters = polygons[~polygons['is_splitter']]
    contains = non_splitters.sjoin(non_splitters, predicate='contains')
    to_kick = contains[~(contains.index == contains['index_right'])]['index_right'].to_list()
    polygons.drop(index=to_kick, inplace=True)
    return polygons


local_crs = 32636

territory = gpd.read_file('project ШлиссельбургMulti.geojson').to_crs(local_crs)
roads = gpd.read_file('roads_shlis.geojson').to_crs(local_crs)
# roads.geometry = roads.geometry.buffer(5, resolution=2)

multipolys = territory_splitter(territory, roads)
multipolys.explore(style_kwds={'fillOpacity': 0.5})

Available workers count 20


In [2]:
from app.gen_planner.python.src import func_zones, GenPlanner

multipolys = multipolys[~multipolys['is_splitter']]
zones, roads = GenPlanner(multipolys).poly2terr2block(func_zones.residential_func_zone)

In [3]:
m1 = multipolys.reset_index().explore(style_kwds={'fillOpacity': 0.2})
zones.explore(m=m1, column='zone_name', tiles='CartoDB positron', cmap='Dark2', categorical=True,
              style_kwds={'fillOpacity': 0.4})
# zones2.explore(m=m1)
m1

In [6]:
local_crs = 32636

lines_orig = multipolys.geometry.apply(geometry_to_multilinestring).to_list()
lines_new = zones.geometry.apply(geometry_to_multilinestring).to_list()
polygons = gpd.GeoDataFrame(geometry=list(polygonize(unary_union(lines_orig + lines_new))),
                            crs=local_crs).explode()
polygons['subsplit_area'] = polygons.area
polygons_points = polygons.copy()
polygons_points.geometry = polygons_points.representative_point()
polygons_points = polygons_points.sjoin(zones, how='inner', predicate='within').drop(columns='index_right')
division = multipolys.sjoin(polygons_points, how='inner', predicate='contains').drop(
    columns=['index_right', 'is_splitter', 'ratio'])

division['zone_str'] = division['zone_name'].apply(lambda x: x.name)
division = division.reset_index().groupby(['index', 'zone_str']).agg(
    {'subsplit_area': 'sum', 'zone_name': 'first', 'geometry': 'first'}).reset_index().set_index(
    'index')
division

,zone_str,subsplit_area,zone_name,geometry
index,,,,
0,business,3.485545e+05,"Territory zone ""business""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
0,recreation,1.856535e+05,"Territory zone ""recreation""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
0,residential,1.488353e+06,"Territory zone ""residential""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
0,transport,1.807672e+05,"Territory zone ""transport""","POLYGON ((389074.006 6643434.599, 389114.274 6..."
1,residential,1.795511e+03,"Territory zone ""residential""","POLYGON ((389074.966 6643470.945, 389076.347 6..."
...,...,...,...,...
76,transport,1.645362e+04,"Territory zone ""transport""","POLYGON ((390682.574 6643834.893, 390650.784 6..."
77,transport,1.444060e+04,"Territory zone ""transport""","POLYGON ((390068.104 6644343.931, 390121.329 6..."
78,transport,1.104738e+04,"Territory zone ""transport""","POLYGON ((390160.325 6644434.762, 390198.441 6..."


In [7]:
import pandas as pd

target_zones = pd.DataFrame.from_dict(func_zones.residential_func_zone.zones_ratio, orient='index',
                                      columns=['zones_ratio']).reset_index().rename(columns={'index': 'zone'})
target_zones['zones_ratio'] = target_zones['zones_ratio'] / target_zones['zones_ratio'].sum()
target_zones['min_area'] = target_zones['zone'].apply(lambda x: x.min_block_area)
target_zones['zone'] = target_zones['zone'].apply(lambda x: x.name)
target_zones.set_index('zone', inplace=True)

possible_areas = division.reset_index().groupby('index').agg(
    {'zone_str': list, 'subsplit_area': list})
possible_areas = possible_areas.join(
    possible_areas.apply(lambda x: pd.Series([y for y in x.subsplit_area], index=x.zone_str), axis=1))
possible_areas = possible_areas.drop(columns=['zone_str', 'subsplit_area'])
possible_areas['poly_area'] = possible_areas[target_zones.index].sum(axis=1)
target_zones['target_area'] = target_zones['zones_ratio'] * possible_areas['poly_area'].sum()
for zone, row in target_zones.iterrows():
    target_zones.loc[zone, 'dif'] = possible_areas[zone].sum() - row['target_area']
target_zones

,zones_ratio,min_area,target_area,dif
zone,,,,
residential,0.555556,160000,3.060854e+06,-77390.328273
business,0.111111,160000,6.121707e+05,78234.554861
recreation,0.111111,320000,6.121707e+05,-6165.881874
transport,0.111111,160000,6.121707e+05,41504.310024
agriculture,0.055556,640000,3.060854e+05,-12712.557969
special,0.055556,160000,3.060854e+05,-23470.096768


In [8]:
possible_areas

,agriculture,business,recreation,residential,special,transport,poly_area
index,,,,,,,
0,NaN,348554.530025,185653.530157,1.488353e+06,NaN,180767.171625,2.203329e+06
1,NaN,NaN,NaN,1.795511e+03,NaN,NaN,1.795511e+03
3,NaN,NaN,NaN,3.034020e+03,NaN,NaN,3.034020e+03
6,NaN,NaN,271201.474824,1.234543e+05,NaN,NaN,3.946558e+05
8,NaN,NaN,54324.423892,1.695264e+05,265221.833521,NaN,4.890726e+05
10,NaN,NaN,NaN,5.481244e+02,645.028322,NaN,1.193153e+03
11,NaN,NaN,NaN,NaN,1923.052665,NaN,1.923053e+03
14,NaN,NaN,NaN,NaN,1118.083118,NaN,1.118083e+03
16,NaN,NaN,NaN,NaN,4682.710734,NaN,4.682711e+03


In [9]:
# Освобождаем "маленькие" полигоны от соперничества, если превышает нужное - регулируем
for poly_ind, poly_row in possible_areas.iterrows():
    valid_indices = poly_row[target_zones.index].notna()
    true_indices = valid_indices[valid_indices].index
    if len(true_indices) == 1:
        continue
    if (target_zones.loc[true_indices, 'min_area'] > poly_row.poly_area).all():
        redistribute = target_zones.loc[true_indices, 'dif'].sort_values(ascending=False)
        zone_to_add = redistribute.index[-1]
        for zone, _ in redistribute.iloc[:-1].items():
            val = poly_row[zone]
            max_change = target_zones.loc[zone_to_add, 'dif']
            if max_change == 0:
                continue

            if val > abs(max_change):
                to_return_val = val - abs(max_change)
                print('max exceed', val - abs(max_change), val)
                mask = possible_areas[zone_to_add].notna() & possible_areas[zone].notna() & (
                        possible_areas[zone_to_add] > 0) & (possible_areas[zone] > 0)
                to_return = possible_areas.loc[mask, true_indices].drop(index=poly_ind)

                for poly_ind2, poly_row2 in to_return.sort_values(by=zone_to_add, ascending=False).iterrows():
                    if possible_areas.at[poly_ind2, zone_to_add] < to_return_val:
                        to_return_val_true = possible_areas.at[poly_ind2, zone_to_add]
                        to_return_val = to_return_val - to_return_val_true
                    else:
                        to_return_val_true = to_return_val
                        to_return_val = 0
                    print(f'from {zone_to_add} to {zone} val {to_return_val_true}, poly {poly_ind2}')
                    target_zones.at[zone, 'dif'] = target_zones.at[zone, 'dif'] + to_return_val_true
                    target_zones.at[zone_to_add, 'dif'] = target_zones.at[zone_to_add, 'dif'] - to_return_val_true

                    possible_areas.at[poly_ind2, zone] = possible_areas.at[poly_ind2, zone] + to_return_val_true
                    possible_areas.at[poly_ind2, zone_to_add] = possible_areas.at[
                                                                    poly_ind2, zone_to_add] - to_return_val_true
                    if to_return_val == 0:
                        break

            print(f'from {zone} to {zone_to_add} val {val}, poly {poly_ind}')
            target_zones.at[zone, 'dif'] = target_zones.at[zone, 'dif'] - val
            target_zones.at[zone_to_add, 'dif'] = target_zones.at[zone_to_add, 'dif'] + val
            possible_areas.at[poly_ind, zone] = possible_areas.at[poly_ind, zone] - val
            possible_areas.at[poly_ind, zone_to_add] = possible_areas.at[poly_ind, zone_to_add] + val
            print()


from special to residential val 645.0283221314968, poly 10

from recreation to special val 2154.1347380237657, poly 17

from transport to recreation val 2079.459835073752, poly 22

from business to transport val 18816.55798446878, poly 29

from business to transport val 30455.14414305088, poly 74

from transport to business val 16785.769922016632, poly 75

from transport to business val 16453.61573446427, poly 76



In [10]:
# Распределяем где 2 конкурирующие зоны

import numpy as np

pos_zones2check = target_zones.index[target_zones['dif'] > 0].to_list()
for max_zone in pos_zones2check:
    neg_zones2check = target_zones.index[target_zones['dif'] < 0].to_list()
    while len(neg_zones2check) > 0:
        max_zone_dif = target_zones.loc[max_zone, 'dif']
        if max_zone_dif <= 0:
            break
        min_zone = target_zones.loc[neg_zones2check, 'dif'].idxmin()
        min_zone_dif = target_zones.loc[min_zone, 'dif']
        if min_zone_dif == 0:
            neg_zones2check.remove(min_zone)
            continue

        other_zones = {max_zone, min_zone} ^ set(target_zones.index)
        zone_inter = possible_areas[
            possible_areas[max_zone].notna() & possible_areas[min_zone].notna() & possible_areas[max_zone] > 0]
        zone_inter = zone_inter[zone_inter[list(other_zones)].isna().all(axis=1)]

        if len(zone_inter) == 0:
            neg_zones2check.remove(min_zone)
            continue

        poly_to_change = zone_inter['poly_area'].idxmin()

        max_give_out = zone_inter.loc[poly_to_change, max_zone]
        to_give_out = min(max_give_out, abs(min_zone_dif), max_zone_dif)
        print(f'from {max_zone} to {min_zone} val {to_give_out}')
        target_zones.at[max_zone, 'dif'] = target_zones.at[max_zone, 'dif'] - to_give_out
        target_zones.at[min_zone, 'dif'] = target_zones.at[min_zone, 'dif'] + to_give_out

        possible_areas.at[poly_to_change, max_zone] = possible_areas.at[poly_to_change, max_zone] - to_give_out
        possible_areas.at[poly_to_change, min_zone] = possible_areas.at[poly_to_change, min_zone] + to_give_out

possible_areas

,agriculture,business,recreation,residential,special,transport,poly_area
index,,,,,,,
0,NaN,348554.530025,185653.530157,1.488353e+06,NaN,180767.171625,2.203329e+06
1,NaN,NaN,NaN,1.795511e+03,NaN,NaN,1.795511e+03
3,NaN,NaN,NaN,3.034020e+03,NaN,NaN,3.034020e+03
6,NaN,NaN,271201.474824,1.234543e+05,NaN,NaN,3.946558e+05
8,NaN,NaN,54324.423892,1.695264e+05,265221.833521,NaN,4.890726e+05
10,NaN,NaN,NaN,1.193153e+03,0.000000,NaN,1.193153e+03
11,NaN,NaN,NaN,NaN,1923.052665,NaN,1.923053e+03
14,NaN,NaN,NaN,NaN,1118.083118,NaN,1.118083e+03
16,NaN,NaN,NaN,NaN,4682.710734,NaN,4.682711e+03


In [46]:
zones_plus = target_zones[target_zones["dif"] > 0]

# Копируем данные для обновления
updated_areas = possible_areas.copy()

for zone, row in zones_plus.sort_values(by='dif').iterrows():
    zones_minus = target_zones[target_zones["dif"] < 0]
    poly_w_neigh = possible_areas[
        possible_areas[zones_minus.index].notna().any(axis=1) & possible_areas[zone].notna() & possible_areas[
            zone] > 0].copy()
    if len(poly_w_neigh) == 0:
        continue
    poly_w_neigh = poly_w_neigh[zones_minus.index.to_list()+[zone,'poly_area']]
    poly_w_neigh['give_away'] = (poly_w_neigh[zone] / poly_w_neigh[zone].sum()) * row.dif
    poly_w_neigh[zones_minus.index] = poly_w_neigh[zones_minus.index].fillna(0)
    print(poly_w_neigh[zones_minus.index])
    poly_w_neigh[zones_minus.index+'_add'] = poly_w_neigh[zones_minus.index].div(poly_w_neigh[zones_minus.index].sum(axis=1),axis=0)
    print(poly_w_neigh)
    break



         residential    agriculture
index                              
6           0.000000  219576.562545
8      274282.489487   34318.250512
         residential    agriculture        special      poly_area  \
index                                                               
6           0.000000  219576.562545  116044.417563  387243.662035   
8      274282.489487   34318.250512  181424.313696  490025.053695   

         give_away  residential_add  agriculture_add  
index                                                 
6      3402.703341         0.000000         1.000000  
8      5319.800222         0.888794         0.111206  


In [48]:
target_zones

,zones_ratio,min_area,target_area,dif
zone,,,,
residential,0.555556,160000,3.064745e+06,-19209.434152
business,0.111111,160000,6.129490e+05,0.000000
recreation,0.111111,320000,6.129490e+05,43402.159348
transport,0.111111,160000,6.129490e+05,18283.416624
agriculture,0.055556,640000,3.064745e+05,-51198.645383
special,0.055556,160000,3.064745e+05,8722.503563


In [ ]:

from app.gen_planner.python.src import GenPlanner, basic_func_zone, func_zones
import geopandas as gpd

zones, roads = GenPlanner(gpd.read_file('project Шлиссельбург.geojson')).poly2terr(func_zones.residential_func_zone)
m1 = zones.explore(column='terr_zone', tiles='CartoDB positron', cmap='Dark2', categorical=True)
roads.explore(m=m1, column='road_lvl', cmap='Dark2', categorical=True)

In [ ]:
t